# Predicting Power Plant Usage 

### Data Set Information:

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant.

A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum is colected from and has effect on the Steam Turbine, he other three of the ambient variables effect the GT performance.



### Attribute Information:

Features consist of hourly average ambient variables
- Temperature (T) in the range 1.81°C and 37.11°C,
- Ambient Pressure (AP) in the range 992.89-1033.30 millibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%
- Exhaust Vacuum (V) in the range 25.36-81.56 cm Hg
- Net hourly electrical energy output (EP) 420.26-495.76 MW
The averages are taken from various sensors located around the plant that record the ambient variables every second. The variables are given without normalization. 

### Data taken from:

https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

Note that the metadata from the previous page had been modified for this demo, however, it would not be difficult to get the xlsx data from the previous page to Python.

### Goal

Predict the electrical power output using machine learning method. This is the column 5.

Jose Nandez, SHARCNET, 2017

## Loading libraries
Load the main libraries/methods that we will use throughout the demo

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression

import pandas as pd 

import matplotlib.pyplot as plt

Make matplotlib to plot everything in our notebook

In [ ]:
%matplotlib inline

## Load the data and visualize it 

In [ ]:
data = pd.read_csv("https://www.sharcnet.ca/~jnandez/power_data.csv")

In [ ]:
data.head()

In [ ]:
#pandas way to plot data, choose the type, the x, and y. Figsize is optional
data.plot(kind = 'scatter', x = 'T', y = 'PE', figsize=(10,8))

In [ ]:
#get some stats
data.describe()

In [ ]:
#get correlations between the columns
data.corr()

In [ ]:
data.plot(kind='scatter',x = 'T', y = 'AP')

## Partition of the data into Training and Testing data sets

In [ ]:
X = data[['T','V','AP','RH']]

In [ ]:
y = data['PE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=100)

In [ ]:
print("Total records: ",len(X))

In [ ]:
print("Total training records: ",len(X_train))

## Linear regression

Let's use all the computer power that we requested (which in this case is only 2 cores, but we can still use them). This is done by means of `multiprocessing`, which it will get the total number of cores in a single node.

In [ ]:
lr = LinearRegression(n_jobs=2)

In [ ]:
lrModel = lr.fit(X_train,y_train)

In [ ]:
lrModel.coef_

In [ ]:
lrModel.predict(X_test)

`score` method returns the coefficient of determination $R^2$ of the prediction.

The coefficient $R^2$ is defined as $(1 - u/v)$, where $u$ is the residual sum of squares `((y_true - y_pred) ** 2).sum()` and $v$ is the total sum of squares `((y_true - y_true.mean()) ** 2).sum()`. The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of $y$, disregarding the input features, would get a $R^2$ score of 0.0.

In [ ]:
print("Linear Regression R2 = ",lrModel.score(X_test,y_test))

In [ ]:
y_pred = lrModel.predict(X_test)

In [ ]:
plt.rc("text",usetex=True)
plt.figure(figsize=(10,8))
plt.scatter(y_test,y_pred)
plt.xlabel(r"$y_{real}$")
plt.ylabel(r"$y_{pred}$");

### Random Forest Regressor

Since a Linear Regression model did a fare job, but not great we could test the Random Forest Regressor and see if this gives a better approach.

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf

In [ ]:
rfModel = rf.fit(X_train,y_train)

In [ ]:
rfModel

In [ ]:
rfModel.feature_importances_ #this will give you an idea which feature has more impact in our model

In [ ]:
print("Random Forest R2 = ",rfModel.score(X_test,y_test))

In [ ]:
y_pred = rfModel.predict(X_test)

In [ ]:
plt.rc("text",usetex=True)
plt.figure(figsize=(10,8))
plt.scatter(y_test,y_pred)
plt.xlabel(r"$y_{real}$")
plt.ylabel(r"$y_{pred}$");

## Cross-Validation (CV) and Grid Search

Cross-Validation (CV) is a common evaluating estimator technique that can help us to overcome **overfitting**. What one does is to take the Training dataset and split it in 2 parts: _training_ and _validating_. Then we train our model with the training dataset, and then we validate it with the validating dataset. We don't touch the testing dataset, since this will be used to `score` our best estimator. This is repetitive, and `sklearn` has function to do this. It uses the `k`-fold. The training set is split into `k` smaller sets, then 

* A model is trained using `k-1` of the folds as training data;
* the resulting model is validated on the remaining part of the data.

Hyper-parameters are parameters that are not directly learnt within estimators. In scikit-learn they are passed as arguments to the constructor of the estimator classes. It is possible and recommended to search the hyper-parameter space for the best cross validation score.

In [ ]:
parameters = {'n_estimators': [10,50,100]}

In [ ]:
rf.set_params(n_jobs = -1)

Create the Grid Search with Cross-Validation

In [ ]:
rfGrid = GridSearchCV(rf, parameters, cv = 5, return_train_score=True) #set k=5 folds

In [ ]:
rfGrid

In [ ]:
%%time
rfModels = rfGrid.fit(X_train,y_train)

In [ ]:
rfModels.cv_results_

### Find the best model

In [ ]:
rfModels.best_estimator_

In [ ]:
bestModel = rfModels.best_estimator_

In [ ]:
print("The best Random Forest Model R2 = ",bestModel.score(X_test,y_test))

In [ ]:
y_pred = bestModel.predict(X_test)

In [ ]:
plt.rc("text",usetex=True)
plt.figure(figsize=(10,8))
plt.scatter(y_test,y_pred)
plt.xlabel(r"$y_{real}$")
plt.ylabel(r"$y_{pred}$");

#### Find a single prediction

In [ ]:
bestModel.predict([[8,70,1006,64]])

## References

* https://www.sharcnet.ca/help/index.php/Graham
* https://docs.computecanada.ca/wiki/Jupyter
* http://scikit-learn.org/stable/
* https://matplotlib.org/
* http://pandas.pydata.org/
* https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant